In [1]:
import json
import os

google_files = [
    'new_york_expanded_restaurants_google.json',
    'san_francisco_expanded_restaurants_google.json',
    'chicago_expanded_restaurants_google.json',
    'boston_expanded_restaurants_google.json',
    'los_angeles_expanded_restaurants_google.json'
]

google_data = []

for file in google_files:
    if os.path.exists(file):
        with open(file, 'r') as f:
            data = json.load(f)
            google_data.extend(data)

with open('google_all_restaurants_combined.json', 'w') as f:
    json.dump(google_data, f, indent=2)

print("✅ Saved to google_all_restaurants_combined.json")


✅ Saved to google_all_restaurants_combined.json


In [3]:
import pandas as pd

# Convert to DataFrame
df = pd.json_normalize(google_data)

# Fix list-type fields
for col in df.columns:
    if df[col].apply(lambda x: isinstance(x, list)).any():
        df[col] = df[col].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)

# Save to CSV
df.to_csv('google_all_restaurants_combined.csv', index=False)

print("✅ Saved to google_all_restaurants_combined.csv")


✅ Saved to google_all_restaurants_combined.csv


In [5]:
import sqlite3

# Ensure column names are valid
df.columns = df.columns.str.strip()                     # Remove spaces
df.columns = df.columns.str.replace(r'\W+', '_', regex=True)  # Replace illegal chars
df.columns = df.columns.str.replace('__+', '_', regex=True)   # Remove double underscores
df.columns = df.columns.str.lower()                    # Lowercase for compatibility

# Drop any unnamed/blank columns
df = df.loc[:, df.columns.notna()]
df = df.loc[:, df.columns != '']

# Save to SQLite
conn = sqlite3.connect('google_restaurants.db')
df.to_sql(name='restaurants', con=conn, if_exists='replace', index=False)
conn.close()

print("✅ Saved to SQLite database: google_restaurants.db (table: restaurants)")


✅ Saved to SQLite database: google_restaurants.db (table: restaurants)


In [7]:
import sqlite3
import pandas as pd

# Connect to database
conn = sqlite3.connect('google_restaurants.db')

# View first few rows
df = pd.read_sql_query("SELECT * FROM restaurants LIMIT 5;", conn)
conn.close()

df.head()


,business_status,formatted_address,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,price_level,rating,...,geometry_location_lat,geometry_location_lng,geometry_viewport_northeast_lat,geometry_viewport_northeast_lng,geometry_viewport_southwest_lat,geometry_viewport_southwest_lng,opening_hours_open_now,plus_code_compound_code,plus_code_global_code,permanently_closed
0,OPERATIONAL,"145 W 53rd St, New York, NY 10019, United States",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,La Grande Boucherie,"{'height': 3600, 'html_attributions': ['<a hre...",ChIJ58OtM8pZwokRbd6DT6gcVys,2.0,4.6,...,40.762634,-73.980836,40.763936,-73.979495,40.761236,-73.982194,0,"Q279+3M New York, USA",87G8Q279+3M,None
1,OPERATIONAL,"9 W 53rd St, New York, NY 10019, United States",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,The Modern,"{'height': 3744, 'html_attributions': ['<a hre...",ChIJMW-gxPtYwokRzDlnsRgq9rA,4.0,4.6,...,40.761081,-73.976753,40.762362,-73.975486,40.759662,-73.978186,0,"Q26F+C7 New York, USA",87G8Q26F+C7,None
2,OPERATIONAL,"1114 6th Ave, New York, NY 10036, United States",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,STK Steakhouse,"{'height': 3024, 'html_attributions': ['<a hre...",ChIJXTDiAwBZwokRHTaHoPYW5oo,3.0,4.8,...,40.754721,-73.982759,40.756116,-73.981549,40.753417,-73.984249,0,"Q238+VV New York, USA",87G8Q238+VV,None
3,OPERATIONAL,"147 W 43rd St, New York, NY 10036, United States",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Tony's Di Napoli,"{'height': 2736, 'html_attributions': ['<a hre...",ChIJVS2qI1VYwokRFo18YsKvHYM,2.0,4.6,...,40.756462,-73.985397,40.757747,-73.984088,40.755047,-73.986788,0,"Q247+HR New York, USA",87G8Q247+HR,None
4,OPERATIONAL,"20 W 40th St, New York, NY 10018, United States",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,La Pecora Bianca Bryant Park,"{'height': 3024, 'html_attributions': ['<a hre...",ChIJKykw-aZZwokRW7K0ykZUxNo,3.0,4.7,...,40.752518,-73.983153,40.753964,-73.981740,40.751264,-73.984440,0,"Q238+2P New York, USA",87G8Q238+2P,None
